In [2]:
import numpy as np 
import tensorflow as tf

import tensorflow_datasets as tfds


# Data

In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to C:\Users\User\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [5]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation = 0.1*mnist_info.splits['train'].num_examples
num_validation = tf.cast(num_validation, tf.int64)

num_test = mnist_info.splits['test'].num_examples
num_test = tf.cast(num_test, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255. 
    
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_trained_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_trained_and_validation_data.take(num_validation)
train_data = shuffled_trained_and_validation_data.skip(num_validation)


BATCH_SIZE = 100 

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation)
test_data = test_data.batch(num_test)


validation_inputs, validation_targets = next(iter(validation_data))

    
    

# Model

In [9]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')  
    
])


## Choose the optimizer and the loss function

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [11]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 4s - loss: 0.3234 - accuracy: 0.9103 - val_loss: 0.1818 - val_accuracy: 0.9443
Epoch 2/5
540/540 - 3s - loss: 0.1371 - accuracy: 0.9586 - val_loss: 0.1166 - val_accuracy: 0.9657
Epoch 3/5
540/540 - 4s - loss: 0.0952 - accuracy: 0.9709 - val_loss: 0.0903 - val_accuracy: 0.9727
Epoch 4/5
540/540 - 3s - loss: 0.0734 - accuracy: 0.9776 - val_loss: 0.0786 - val_accuracy: 0.9762
Epoch 5/5
540/540 - 4s - loss: 0.0568 - accuracy: 0.9824 - val_loss: 0.0817 - val_accuracy: 0.9777


# Test the model

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0960 - accuracy: 0.9716


In [13]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 97.16%
